In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from Planet.data import PlanetDataCollection
from Planet.scheduler import ParamScheduler
from Planet.utils import get_transforms, get_df
from Planet.train import NNTrainer 
from torchvision import models
from pathlib import Path

In [3]:
import torch
from torch import optim
from functools import partial

In [4]:
path = Path('/home/anukoolpurohit/Documents/AnukoolPurohit/Datasets/Planet')

In [5]:
tfms = get_transforms()

In [6]:
data = PlanetDataCollection.from_csv(path, 'train_v2.csv', 'train-jpg',
                           tfms, name_col='image_name', label_col='tags',
                           delimter=' ',bs=64,test_folder='test-jpg')

In [7]:
arch = models.resnet50

In [8]:
def annealer(f):
    def _inner(start, end): 
        return partial(f, start, end)
    return _inner

In [9]:
@annealer
def sched_lin(start, end, pos):
    return start + pos*(end-start)

In [122]:
import math
from typing import Iterable
from torch import tensor

In [119]:
def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

In [115]:
@annealer
def sched_cos(start, end, pos):
    return start + (1 + math.cos(math.pi*(1-pos))) * (end-start)/2
@annealer
def sched_no(start, end, pos):
    return start
@annealer
def sched_exp(start, end, pos):
    return start * (end/start) ** pos

In [125]:
def combine_scheds(pcts, scheds):
    assert sum(pcts) == 1.
    pcts = tensor([0] + listify(pcts))
    assert torch.all(pcts >=0)
    pcts = torch.cumsum(pcts, 0)
    def _inner(pos):
        idx = (pos >= pcts).nonzero().max()
        actual_pos = (pos - pcts[idx]) / (pcts[idx+1] - pcts[idx])
        return scheds[idx](actual_pos)
    return _inner

In [126]:
sched = combine_scheds([0.3, 0.7], [sched_cos(3e-4, 1e-2), sched_cos(1e-2, 1e-4)])

In [127]:
trainer = NNTrainer(data,arch,sched, optim.Adam)

In [128]:
trainer.fit(3)

       Epoch            Training Loss       Validation Loss   
         0                 0.174717             0.153029      


         1                 0.156217             0.130204      


         2                 0.137392             0.122671      


In [ ]:
trainer.log.plot_loss()

In [ ]:
trainer.log.plot_lr()

In [ ]:
a.append(3)

In [ ]:
a

In [ ]:
from torch import tensor

In [ ]:
tensor(0.)

In [38]:
from tqdm.auto import tqdm

In [39]:
class ProgessBar2(tqdm):
    def __init__(self, trainer):
        super().__init__()
        self.trainer = trainer
        return
    
    def progress_bar(self, itr):
        self.bar = tqdm(itr, leave=False)
        stage = "Training " if self.trainer.in_train else "Validation"
        self.bar.set_description(f"Epoch-{self.trainer.epoch}: {stage}")
        loss = self.trainer.loss.detach().item()
        self.bar.set_postfix({f"Loss":"{:.2f}".format(loss)})
        return self.bar
    
    def update(self):
        super().update()
        loss = self.trainer.loss.detach().item()
        self.bar.set_postfix({f"Loss":"{:.2f}".format(loss)})
        self.bar.update()

In [40]:
tqdm.

tqdm._tqdm_notebook.tqdm_notebook